## Problem set 1
In class work for 1/16/2025.

In [1]:
import numpy as np

In [3]:
# define some constants (SI units)
hbar = 1.0546e-34 # J s, reduced Planck constant
ec = 1.609e-19 # C, proton charge
m_e = 9.109e-31 # kg, electron mass
g = 2.002 # electron gyromagnetic factor
mu_B = ec*hbar/(2*m_e) # J/T, Bohr magneton
eV = 1.609e-19 # J , electron volt definition

In [4]:
# some other useful definitions
# Spin-1/2 matrices
Sz = hbar/2*np.matrix([[1,0],[0,-1]])
Sx = hbar/2*np.matrix([[0,1],[1,0]])
Sy = hbar/2*(1.0j)*np.matrix([[0,-1],[1,0]])

In [6]:
print(Sx)
print(Sy)
print(Sz)

[[0.000e+00 5.273e-35]
 [5.273e-35 0.000e+00]]
[[0.+0.000e+00j 0.-5.273e-35j]
 [0.+5.273e-35j 0.+0.000e+00j]]
[[ 5.273e-35  0.000e+00]
 [ 0.000e+00 -5.273e-35]]


### Problem 1
An electron is in a magnetic field of $\mathbf{B} = B_1 \hat{x} + B_0 \hat{z}$.
- What is the Hamiltonian for this system?  (See McIntyre Section 3.2) Express this as a matrix in the basis of the $z$ component of spin ($S_z$).
- Suppose the magnetic field has (in SI units) $B_0=50\,\mathrm{mT}, B_1=10\,\mathrm{mT}$.
What are the energy eigenvalues (in units of electron-volts, eV) and eigenstates for this system?  Express the eigenstates as vectors in the $S_z$ basis.
Give all numerical values to 3 significant digits.
- The particle has an initial state of $\ket{+_z}$.  What is the state of the particle as a function of time, $\ket{\psi(t)}$?  (The spin will precess.  Calculate the angular frequency $\omega$ of this precession in units of rad/s.)
- Qualitatively, how does the expectation value of the particle's spin vector vary as a function of time, $\left<\mathbf{S}(t)\right>$?


In [2]:
# First define the magnetic field (using units of T)
Bx = 10e-3
Bz = 50e-3
By = 0.0
B = np.matrix([[Bx,By,Bz]]).T # we make a 1x3 matrix, then take its transpose to get a 3x1 matrix, i.e. column vector.
print(B)

[[0.01]
 [0.  ]
 [0.05]]


From McIntyre Eq. (3.23), the Hamiltonian is
$$
H = \frac{e}{m_e} \vec{S}\cdot\vec{B}
$$

In [8]:
# The Hamiltonian
# From McIntyre Eq. (3.23)
H = ec/m_e*(Sx*Bx + Sy*By + Sz*Bz)
print(H)

[[ 4.65707377e-25+0.j  9.31414755e-26+0.j]
 [ 9.31414755e-26+0.j -4.65707377e-25+0.j]]


In [9]:
# initial state
psi0 = np.matrix([[1,0]]).T # column vector
print(psi0)

[[1]
 [0]]


### Bonus, time permitting
Let's redo this problem using a set of "natural units" to make the numerical values closet to one. (This helps avoid rounding errors in numerical calculations.)
Generally, we would pick some combination of constants to find typical values of length, mass, and time, and use those to define a new set of units within which to work the problem.

In this case, there is already a standard choice of natural units, called "atomic units" or "Hartree units".  The units are chosen to force the following:
$$
\begin{aligned}
e &= 1 &&\text{elementary charge} \\
m_e &=1 &&\text{electron mass} \\
\hbar &=1 &&\text{reduced Planck constant} \\
k_e = \frac{1}{4\pi\epsilon_0}&=1 &&\text{Coulomb force constant}
\end{aligned}
$$

With this choice, we get the following new units for length and time (the mass unit is the electron mass, above):
$$
\begin{aligned}
m_e &= 9.109\times 10^{-31}\,\mathrm{kg} && \text{electron mass}\\
a_0 =  &= 5.292\times 10^{-11}\,\mathrm{m} && \text{Bohr radius of the H atom} \\
t_0 &= 2.419\times 10^{-17}\,\mathrm{s} && \text{1/Hartree frequency}.
\end{aligned}
$$

Let's rework the problem using these new units.  What do you notice about the numerical values?

## Problem 2
A particle with mass $m$ is in an infinite square well of width $L$, with its lower edge at $x=0$.
- What is the Hamiltonian operator for this system?  Express it in the position representation.

The particle is in the initial state
$\psi(x,0) = C \sin^3 (\pi x/L)$ for the interval $0<x<L$ and zero elsewhere.

- Find the normalization constant $C$ and express $\psi(x,0)$ in terms of the eigenstates.  (Hint: there are at least two ways to do this: integrating directly or by expressing the initial wavefunction as a superposition of eigenstates.  See if you can do both.)
- What is the wavefunction at a later time $\psi(x,t)$?
